# TEORÍA LINEAL -  SOLUCIÓN DE SISTEMAS DE TUBERÍAS

En este notebook se mostrará cómo se soluciona una red abierta que conecta cuatro tanques con un nodo central. Se conocen las alturas de cada uno de los tanques respecto a un datum y las características físicas de cada una de las tuberías que hacen parte del sistema. Todo parte de la ecuación básica de pérdidas en una tubería:

$$h=CQ^n$$

Donde $h$ es la pérdida de energía en el sistema, $Q$ el caudal y $n$ el exponente de la ecuación de pérdidas que se esté utilizando ($n=1.85$ para el caso de la ecuación de Hazen-Williams y $n=2$ cuando se habla de la ecuación de Darcy-Weisbach). Para este caso particular, se empleará una versión un poco diferente que permite ponerle un signo al caudal, de acuerdo a su dirección de flujo relativa a los nodos que se están analizando. De esta manera la ecuación se transforma en: 

$$h=C|Q^{n-1}|Q$$

## Ecuaciones de gobierno

Las ecuaciones de gobierno que se usarán para resolver el sistema son la consrvación de la masa en nodos y conservación de energía en las tuberías del sistema. En general, se cumple que el número de ecuaciones que se pueden proponer son iguales al número de incógnitas que tiene el sistema. Entonces, en los nodos deberá cumplirse que: 

$$\sum_{nodo}Q = 0$$

Ahora, para las conservaciones de energía se asumirá que todos los caudales salen del nodo y se dirigen al tanque (en la convención usual de la hidráulica, todos los caudales son positivos). Entonces, las conservaciones de energía para cada nodo se pueden expresar por medio de la ecuación de Bernoulli como: 

$$H_J=H_i+C_i|Q_i|^{n-1}Q_i$$

Donde $H_J$ y $H_i$ son las alturas piezométricas del nodo que se está analizando y el tanque donde termina la tubería $i$, $C_i$ la constante de la ecuación de pérdidas para la tubería $i$ y $Q_i$ el caudal de dicha tubería. Dejando a la izquierda las incógnitas de la ecuación y a la derecha el término conocido de la altura del tanque, la ecuación de energía en general queda:

$$H_J-C_i|Q_i|^{n-1}Q = H_i$$

## Parte operativa del método

Para poder implmentar el método de la teoría lineal, se necesita suponer los caudales de cada uno de los nodos (no importa si cumplen o no conservación de masa en el nodo). Luego, se calculan los productos $C_i|Q_i|^{n-1}$ y se monta un sistema lineal de ecuaciones, cuyas incógnitas serán los caudales $Q_i$ y la altura piezométrica del nodo $H_J$. Cuando se resuelve el sistema, se comparan las suposiciones con los resultados obtenidos. Si son iguales, i.e. están en un nivel de tolerancia aceptable, se habrá obtenido la respuesta definitiva. De lo contrario, los resultados se usan para recalcular los $C_i|Q_i|^{n-1}$ y se repite el proceso tantas veces como sea necesario. 

In [1]:
import numpy as np

# Alturas de los tanques del problema en metros de elevación
H = np.array([24, 2, 12, 6])

# Longitudes de cada una de las tuberías
L = np.array([1800, 2400, 1200, 2400], dtype = 'float64')

# Diámetros de las tuberías en mm
D = np.array([500, 600, 400, 900], dtype = 'float64')

# Coeficientes de hazen para los tubos
Ch = np.array([130, 130, 130, 120], dtype = 'float64')

# Pasando los diámetros a metros para los cálculos de la constante
D /= 1000

### Calculando constante de la ecuación de Hazen-Williams

Se usará la ecuación de Hazen-Williams para pérdidas y se aprovechará para determinar la forma de dicha ecuación cuando se necesita hallar la pérdida, en vez de la velocidad o el caudal. Partimos de: 

$$\bar{V}=0.849C_HR_h^{0.63}S_f^{0.54}$$

Donde $\bar{V}$ es la velocidad promedio en la tubería, $C_H$ es el coeficiente de Hazen representativo para el material de la tubería, $R_h$ es el radio hidráulico de la sección analizada y $S_f$ es la pendiente de la línea de energía, que se puede expresar como $h_f/L$. Para hallar el caudal, se multiplica por el área de la sección transversal a ambos lados, y se reemplaza el radio hidráulico por $D/4$, ya que se tiene tuberías redondas. 

$$Q=0.849\frac{\pi D^{2.63}C_H}{4^{1.63}}\Bigg(\frac{h_f}{L}\Bigg)^{0.54}$$

Calculando los términos constantes: 

$$Q=0.2784 C_H D^{2.63}\Bigg(\frac{h_f}{L}\Bigg)^{0.54}$$

Ahora, queremos despejar las pérdidas $h_F$ de la ecuación de caudal.

$$h_f=L\Bigg(\frac{Q}{0.2784C_H D^{2.63}}\Bigg)^{1.85}$$

Simplificando lo que se puede y dejando todo como se requiere: 

$$h_f = 10.674 L \Bigg(\frac{Q}{C_H D^{2.63}}\Bigg)^{1.85}$$

La siguiente caja de código ilustra el resultado de las operaciones llevadas a cabo. 

In [2]:
# Cálculo de constante de Hazen-Williams
const = 0.849 * np.pi / 4 ** 1.63
print('El valor de la constante para la ecuación de caudal es: %.4f'%(const))

expon = 1 / 0.54
print('El valor del exponente de Hazen-Williams es: %.4f'%(expon))

const2 = (1 / const) ** expon
print('La constante que acompaña la ecuación de pérdidas es: %.4f'%(const2))

El valor de la constante para la ecuación de caudal es: 0.2784
El valor del exponente de Hazen-Williams es: 1.8519
La constante que acompaña la ecuación de pérdidas es: 10.6742


A continuación se enumeran los pasos del método de teoría lineal para la solución de redes. Este método es replicable para cualquier tipo de red que se desee calcular. 

1. Contar las incógnitas que hay en el sistema
2. Plantear las ecuaciones de conservación necesarias y con ellas ensamblar un sistema matriz-vector
3. Asumir caudales iniciales y calcular las constantes $C_i|Q_i|^{n-1}$.
4. Resolver el sistema y verificar resultados
5. Reemplazar los valores obtenidos en la ecuación y volver al paso 3

In [3]:
# Construyendouna función que imprime de forma medianamente ordenada el resultado de la solución del 
# sistema matriz vector propuesto
def imprimir_resultado(X):

    # Mensaje
    msg = ["Altura de nodo (m): ", "Caudal "]

    # Imprimiendo el mensaje en un loop
    for i, numero in enumerate(X):

        if i == 0: print(msg[i] + str(round(X[0], 4)))

        else:
            sello = msg[1] + ' ' + str(i) + ' (m3/s): ' + str(round(X[i], 4))
            print(sello)

In [4]:
# Calculando el vector de constantes C que tiene cada una de las tuberías
C = const2 * L / (Ch * D ** 2.63) ** expon

# Asumiendo caudales iniciales en cada una de las tuberías
Q0 = np.array([0.2] * 4)

# Calculando C |Q|^0.85
K = C * np.abs(Q0) ** (expon - 1)

# Vector de mano derecha del sistema - el 0 corresponde al resultado de consevación de masa en el nodo
# los valores de la altura de los tanques corresponden al resultado de las ecuaciones de conservación 
# de energía
b = np.concatenate(([0], H))

# Ensamblando la matriz del sistema
A = np.zeros([len(H) + 1, len(H) + 1])

# Planteando conservación de la masa en la primera fila de la matriz
A[0, 1:] = 1

# Para conservación de energía, la primera coumna de la matriz tiene 1, a excepción de la 
# primera fila
A[1:, 0] = 1

# Los valores de la constante K van en la diagonal principal de la matriz desde la segunda fila
# en adelante. Esto construye las ecuaciones de conservación de energía
for i in range(1, len(b)): A[i, i] = -K[i - 1]

print('Así queda la matriz del sistema lineal que se pretende resolver: ')
print(np.round(A, 3))

Así queda la matriz del sistema lineal que se pretende resolver: 
[[  0.      1.      1.      1.      1.   ]
 [  1.    -17.362   0.      0.      0.   ]
 [  1.      0.     -9.526   0.      0.   ]
 [  1.      0.      0.    -34.316   0.   ]
 [  1.      0.      0.      0.     -1.533]]


In [5]:
# Resolviendo el sistema lineal de ecuaciones por primera vez: 
X = np.linalg.solve(A, b)

# LLamando la función que imprime el resultado
imprimir_resultado(X)

Altura de nodo (m): 6.9381
Caudal  1 (m3/s): -0.9827
Caudal  2 (m3/s): 0.5184
Caudal  3 (m3/s): -0.1475
Caudal  4 (m3/s): 0.6118


In [ ]:
# Implementando todo en un ciclo que resuelva hasta llegar a una solución 
# estable:
err = 1e10                           # Valor inicial para el error (muy alto)
tol = 1e-4                           # Valor de tolerancia del solver
count = 1                            # Contador de veces que se itera

while err > tol:

    # Calcular los K nuevos con los caudales encontrados en la primera iteración
    K = C * np.abs(X[1:]) ** (expon - 1)

    # Reemplazando los valores de K en la matriz
    for i in range(1, len(b)): A[i, i] = -K[i - 1]

    # Solucionando el sistema
    X1 = np.linalg.solve(A, b)

    # Calculando el error
    err = np.linalg.norm(X - X1)

    # Imprimiendo el resultado
    print('\nIteración %i \t Error: %.3e'%(count, err))
    imprimir_resultado(X1)

    # Reemplazando valores 
    count += 1
    X = (X1 + X) / 2 # Se promedia para evitar oscilaciones bruscas 


Iteración 1 	 Error: 8.803e-01
Altura de nodo (m): 6.8755
Caudal  1 (m3/s): -0.2541
Caudal  2 (m3/s): 0.2274
Caudal  3 (m3/s): -0.1935
Caudal  4 (m3/s): 0.2203

Iteración 2 	 Error: 3.345e-01
Altura de nodo (m): 6.7493
Caudal  1 (m3/s): -0.3798
Caudal  2 (m3/s): 0.2933
Caudal  3 (m3/s): -0.1753
Caudal  4 (m3/s): 0.2618

Iteración 3 	 Error: 1.042e-01
Altura de nodo (m): 6.7391
Caudal  1 (m3/s): -0.4562
Caudal  2 (m3/s): 0.3222
Caudal  3 (m3/s): -0.1736
Caudal  4 (m3/s): 0.3075

Iteración 4 	 Error: 4.496e-02
Altura de nodo (m): 6.7389
Caudal  1 (m3/s): -0.4736
Caudal  2 (m3/s): 0.3267
Caudal  3 (m3/s): -0.1733
Caudal  4 (m3/s): 0.3201

Iteración 5 	 Error: 2.234e-02
Altura de nodo (m): 6.7389
Caudal  1 (m3/s): -0.4753
Caudal  2 (m3/s): 0.3271
Caudal  3 (m3/s): -0.1733
Caudal  4 (m3/s): 0.3215

Iteración 6 	 Error: 1.117e-02
Altura de nodo (m): 6.7389
Caudal  1 (m3/s): -0.4754
Caudal  2 (m3/s): 0.3271
Caudal  3 (m3/s): -0.1733
Caudal  4 (m3/s): 0.3216

Iteración 7 	 Error: 5.583e-03
Al